In [1]:
import torch
from torchvision.models import resnet34

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

model = resnet34(pretrained=False)
print(count_parameters(model))
model

In [12]:
model.fc.out_features = torch.nn.Linear(512, 8)
print(count_parameters(model))
model.fc

21811184


Linear(
  in_features=512, out_features=Linear(in_features=512, out_features=8, bias=True), bias=True
  (out_features): Linear(in_features=512, out_features=8, bias=True)
)

In [11]:
model.conv1 = torch.nn.Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
rnd = torch.rand([2,6,256,256])
model(rnd)
# model.conv1
model

ResNet(
  (conv1): Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
eye = torch.eye(3)
rnd = torch.rand([3,3])

eye.mul(rnd)

tensor([[0.1112, 0.0000, 0.0000],
        [0.0000, 0.2509, 0.0000],
        [0.0000, 0.0000, 0.8431]])

In [20]:
rnd_inv = rnd.inverse()
rnd.matmul(rnd_inv)

eye = torch.eye(3, dtype=rnd.dtype, device=rnd.device, requires_grad=True)
eye

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], requires_grad=True)

In [2]:
def consistency_loss(h_ab: torch.Tensor, h_ba: torch.Tensor):
        loss = torch.nn.functional.mse_loss(h_ab.matmul(h_ba), torch.eye(3, dtype=rnd.dtype, device=rnd.device))
        return loss

rnd = torch.rand([3,3])
rnd_inv = torch.rand([3,3], requires_grad=True)

print(rnd.matmul(rnd_inv))

opt = torch.optim.Adam([rnd_inv], lr=0.1)

for _ in range(1000):
    opt.zero_grad()
    loss = consistency_loss(rnd, rnd_inv)
    loss.backward()
    opt.step()
    # print(loss)
print(rnd.matmul(rnd_inv))

tensor([[ 1.0000e+00,  7.4371e-09, -3.7514e-07],
        [-1.1909e-08,  1.0000e+00,  3.7465e-07],
        [ 7.9462e-08,  2.5468e-07,  1.0000e+00]], grad_fn=<MmBackward>)


In [47]:
def content_loss(f_a: torch.Tensor, f_b: torch.Tensor, m_a: torch.Tensor, m_b: torch.Tensor):
    eps = torch.finfo(f_a.dtype).eps
    loss = torch.sum(m_a.mul(m_b).mul(torch.abs(f_a.sub(f_b)))).div(torch.sum(m_a.mul(m_b) + eps))
    return loss

f_a = torch.rand([3,1,3,3])
f_b = torch.rand([3,1,3,3])
m_a = torch.eye(3)
m_a = m_a.reshape(1,1,3,3)
m_a = m_a.repeat(3,1,1,1)
m_b = m_a

eps = torch.finfo(f_a.dtype).eps
torch.sum(m_a.mul(m_b).mul(torch.abs(f_a.sub(f_b)))).div(torch.sum(m_a.mul(m_b) + eps))

tensor(0.4430)

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=0, activation: F=F.relu):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.norm = nn.BatchNorm2d(num_features=out_channels)
        self.activation = activation

    def forward(self, x):
        x = self.conv(x)
        x = self.norm(x)
        x = self.activation(x)
        return x

conv = ConvBlock(1,3,activation=F.sigmoid)
rnd = torch.rand([3,1,3,3])

conv(rnd)

tensor([[[[0.1980]],

         [[0.7039]],

         [[0.3972]]],


        [[[0.6277]],

         [[0.6305]],

         [[0.7977]]],


        [[[0.7061]],

         [[0.1978]],

         [[0.2780]]]], grad_fn=<SigmoidBackward>)